In [1]:
from pathlib import Path
from IPython.display import Markdown
import re

import papermill as pm

from src import paths

In [2]:
ORG_NAME = 'Decentraland'
ORG_PLATFORM = 'snapshot'
SPLITS_FREQ: str = 'W-THU' # split weekly
SPLITS_NORMALIZE = True # Wether or not to move everything to 00:00
LAST_SPLITS = 10 # Use just last 10 splits

In [3]:
# Display the diagram from the README.md

README = Path('./README.md')

with open(README, 'r') as f:
    diagrams = re.findall(r'```mermaid[\s\S]*```', f.read())

Markdown("\n".join(diagrams))

```mermaid
graph LR
10_baseline --> 20_results

10_baseline --> 11_pln-tune -.-> 12_hybrid
07_microsoft_tuning --> 09_analyze_results --> 12_hybrid
12_hybrid --> 20_results
10_baseline --> 09_analyze_results
```

## Define some common things

In [27]:
def run_notebook(name):
    assert Path(name).exists(), f"No existe el fichero {name}"
    
    outpath = Path(f'./nbout/{ORG_NAME}')
    outpath.mkdir(parents=True, exist_ok=True)

    params = pm.inspect_notebook(name)
    for p in ['ORG_NAME', 'ORG_PLATFORM', 'SPLITS_FREQ', 'SPLITS_NORMALIZE', 'LAST_SPLITS']:
        assert p in params, f'{p} is not in notebook params'
    
    return pm.execute_notebook(
        name, 
        outpath/name,
        autosave_cell_every=60,
        parameters=dict(
            ORG_NAME = ORG_NAME,
            ORG_PLATFORM = ORG_PLATFORM,
            SPLITS_FREQ = SPLITS_FREQ,
            SPLITS_NORMALIZE = SPLITS_NORMALIZE,
            LAST_SPLITS = LAST_SPLITS,
        ),
    )

# run_notebook('11_pln-tune.ipynb', 'snapshot', 'Decentraland')

In [17]:
pm.inspect_notebook('11_pln-tune.ipynb')['MDF_FILE']

{'name': 'MDF_FILE',
 'inferred_type_name': 'None',
 'default': 'paths.pln_mdf(ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE)',
 'help': ''}

## Run baseline

In [18]:
#papermill_description=Baseline
run_notebook('10_baseline_mp.ipynb')

Executing:   0%|          | 0/47 [00:00<?, ?cell/s]

## Run pln_tune

In [19]:
#papermill_description=pln_tune
# if paths.pln_mdf(ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE).exists():
#     print(f"Skipping {ORG_NAME} because it has been done")    
run_notebook('11_pln-tune.ipynb')

Executing:   0%|          | 0/59 [00:00<?, ?cell/s]

2024-02-28 11:02:00.147035: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 11:02:00.147057: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 11:02:00.147074: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Run microsoft_tuning

In [8]:
#papermill_description=microsoft
run_notebook('07_microsoft_tuning.ipynb')

Executing:   0%|          | 0/26 [00:00<?, ?cell/s]

2024-02-28 10:46:36.367498: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 10:46:36.367521: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 10:46:36.367537: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
*** SIGTERM received at time=1709117220 on cpu 16 ***
PC: @     0x7f1b01216e2e  (unknown)  epoll_wait
    @     0x7f1b01133520  1013368720  (unknown)
    @     0x7f1b00d3922c  (unknown)  (unknown)
[2024-02-28 10:47:00,347 E 95718 95718] logging.cc:361: *** SIGTERM received at time=1709117220 on cpu 16 ***
[2024-02-28 10:47:00,347 E 95718 95718] logging.cc:361:

## Run analyze_results

In [28]:
#papermill_description=analyze
nb = run_notebook('09_analyze_results.ipynb')

Executing:   0%|          | 0/41 [00:00<?, ?cell/s]

## Run hybrid

In [10]:
#papermill_description=hybrid
run_notebook('12_hybrid.ipynb')

Executing:   0%|          | 0/39 [00:00<?, ?cell/s]

2024-02-28 10:47:12.241700: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 10:47:12.241723: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 10:47:12.241737: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
